# Notebook for identifying and removing bottlenecks from ICET implementation

In [41]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
import time

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 4*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
from ICET_spherical import ICET
from utils import R_tf
from metpy.calc import lat_lon_grid_deltas

%load_ext autoreload
%autoreload 2
%autosave 180
# %matplotlib notebook

# %%bash
# # python -m cProfile scan_match.py
# python scan_match.py

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Autosaving every 180 seconds


In [13]:
basepath = '/media/derm/06EF-127D1/KITTI'
# sequence = '03' #forest
sequence = '09' #trees and small town
dataset = pykitti.odometry(basepath, sequence)
velo1 = dataset.get_velo(400)
c1 = velo1[:,:3]
velo2 = dataset.get_velo(401)
c2 = velo2[:,:3]

it = ICET(cloud1 = c1, cloud2 = c2, fid = 70, niter = 9, 
           draw = False, group = 2, RM = True, DNN_filter = False)

Ground truth poses are not avaialble for sequence 09.

 loading model took 2.384185791015625e-07 
 total:  3.5762786865234375e-06

 converting to spherical took 0.029747486114501953 
 total:  0.029764413833618164

 getting spherical grid 1.4626104831695557 
 total:  1.4923903942108154

 fit_gaussian for scan 1 0.22727727890014648 
 total:  1.7197356224060059

 ~~~~~~~~~~~~~~ 
 fit_gaussian for scan 2 0.20335674285888672 
 total:  1.9235024452209473 
 ~~~~~~~~~~~~~~

 estimated solution vector X: 
 tf.Tensor([ 0.32104814  0.01419096  0.00261805 -0.00059314  0.00140314 -0.00500818], shape=(6,), dtype=float32)

 ~~~~~~~~~~~~~~ 
 correcting solution estimate 0.01730966567993164 
 total:  1.9408438205718994 
 ~~~~~~~~~~~~~~

 ~~~~~~~~~~~~~~ 
 fit_gaussian for scan 2 0.19917750358581543 
 total:  2.1402862071990967 
 ~~~~~~~~~~~~~~

 estimated solution vector X: 
 tf.Tensor(
[ 6.8457842e-01  1.3623415e-03  8.6901868e-03 -4.4888968e-04
  1.6689554e-03  2.4430999e-03], shape=(6,), dtype=float3

# fit_gaussian()

In [189]:
def fg2(cloud, rag, npts):
    """new method of fitting gaussian to better handle ragged input data"""

    coords = tf.gather(cloud, rag)
    mu = tf.math.reduce_mean(coords, axis = 1)[:,None]
    #     print(mu)
    #     print(coords)
    
    xpos = tf.gather(cloud[:,0], rag)
    ypos = tf.gather(cloud[:,1], rag)
    zpos = tf.gather(cloud[:,2], rag)
#     c = tfp.stats.covariance(xpos.to_tensor(), ypos.to_tensor())
    
    #old and slow
#     xx = tf.math.reduce_sum(tf.math.square(xpos - mu[:,:,0] ), axis = 1)/npts

    #TODO: try randomly sampling 30 points from each ragged cell, use reduced num pts to calculate cov

    nraggeds = tf.shape(coords)[0].numpy()
    for i in range(nraggeds):
        c  = tfp.stats.covariance(coords[i])
    
    mu = None
    sigma = None
    return(mu, sigma)

In [190]:
s = time.time()
mu2, sigma2 = it.fit_gaussian(it.cloud2_tensor, it.inside2, tf.cast(it.npts2, tf.float32))
print("\n took", time.time() - s, " s with old method")

s = time.time()
mu2, sigma2 = fg2(it.cloud2_tensor, it.inside2, tf.cast(it.npts2, tf.float32))
print(" \n took", time.time() - s, " s with new method")

# print(it.npts2)
# print(it.inside2)


 took 0.1895885467529297  s with old method
 
 took 1.549006462097168  s with new method
